Deep Learning
=============

Assignment 6
------------

After training a skip-gram model in `5_word2vec.ipynb`, the goal of this notebook is to train a LSTM character model over [Text8](http://mattmahoney.net/dc/textdata) data.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve

ImportError: Traceback (most recent call last):
  File "C:\Users\Tiki\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 41, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "C:\Users\Tiki\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 21, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "C:\Users\Tiki\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 20, in swig_import_helper
    return importlib.import_module('_pywrap_tensorflow_internal')
  File "C:\Users\Tiki\Anaconda3\envs\tensorflow\lib\importlib\__init__.py", line 37, in import_module
    __import__(name)
ImportError: No module named _pywrap_tensorflow_internal


Failed to load the native TensorFlow runtime.

See https://www.tensorflow.org/install/install_sources#common_installation_problems

for some common reasons and solutions.  In

In [14]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified %s' % filename)
  else:
    print(statinfo.st_size)
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip


In [15]:
def read_data(filename):
  with zipfile.ZipFile(filename) as f:
    name = f.namelist()[0]
    data = tf.compat.as_str(f.read(name))
  return data
  
text = read_data(filename)
print('Data size %d' % len(text))

Data size 100000000


Create a small validation set.

In [16]:
valid_size = 1000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print(train_size, train_text[:64])
print(valid_size, valid_text[:64])

99999000 ons anarchists advocate social relations based upon voluntary as
1000  anarchism originated as a term of abuse first used against earl


Utility functions to map characters to vocabulary IDs and back.

In [17]:
vocabulary_size = len(string.ascii_lowercase) + 1 # [a-z] + ' '
first_letter = ord(string.ascii_lowercase[0]) # return an integer

def char2id(char):
  if char in string.ascii_lowercase:
    return ord(char) - first_letter + 1
  elif char == ' ':
    return 0
  else:
    print('Unexpected character: %s' % char)
    return 0
  
def id2char(dictid):
  if dictid > 0:
    return chr(dictid + first_letter - 1)
  else:
    return ' '

print(char2id('a'), char2id('z'), char2id(' '), char2id('ï'))
print(id2char(1), id2char(26), id2char(0))

Unexpected character: ï
1 26 0 0
a z  


Function to generate a training batch for the LSTM model.

In [18]:
batch_size=64
num_unrollings=10

class BatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size // batch_size
    self._cursor = [ offset * segment for offset in range(batch_size)]
    self._last_batch = self._next_batch()
  
  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.float)
    for b in range(self._batch_size):
      # the consecutive characters in the batch are segment away in the text
      batch[b, char2id(self._text[self._cursor[b]])] = 1.0
      self._cursor[b] = (self._cursor[b] + 1) % self._text_size
    return batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in range(self._num_unrollings):
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches

def characters(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (most likely) character representation."""
  return [id2char(c) for c in np.argmax(probabilities, 1)]

def batches2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  s = [''] * batches[0].shape[0]
  for b in batches:
    s = [''.join(x) for x in zip(s, characters(b))] # join individual character together
    print(s)
  return s

train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 1, 1)

print(batches2string(train_batches.next()))
print(batches2string(train_batches.next()))
print(batches2string(valid_batches.next()))
print(batches2string(valid_batches.next()))

['o', 'w', 'l', ' ', 'm', 'h', 'y', 'a', 't', 'm', 'n', 'h', 'e', 'e', 'o', 'y', 'o', 'a', ' ', 'a', 'i', ' ', 't', 'd', 'f', 'a', 'e', 'e', 'a', 'r', 'i', 'o', 'a', 'g', 'i', 'r', 'c', 'a', ' ', 'm', 't', 'u', 'e', 'o', 'o', 's', 'k', 'e', 'w', 'e', 't', 'e', ' ', 'i', 't', 'd', 't', 'e', 'f', 'd', 't', 'a', 'a', 's']
['on', 'wh', 'll', ' a', 'ma', 'he', 'y ', 'ay', 'ti', 'mi', 'ne', 'he', 'e ', 'eb', 'o ', 'ye', 'or', 'a ', ' t', 'ar', 'it', ' a', 'ti', 'dy', 'f ', 'at', 'e ', 'en', 'am', 'rv', 'io', 'o ', 'a ', 'gh', 'in', 'ro', 'ca', 'as', ' d', 'mo', 't ', 'u ', 'e ', 'o ', 'of', 's ', 'kl', 'er', 'ws', 'et', 'th', 'et', ' s', 'is', 'ti', 'd ', 'th', 'en', 'fe', 'du', 'tr', 'at', 'ap', 'si']
['ons', 'whe', 'lle', ' ab', 'mar', 'hel', 'y a', 'ay ', 'tio', 'mig', 'new', 'he ', 'e l', 'ebe', 'o b', 'yer', 'ore', 'a f', ' tw', 'ari', 'ity', ' an', 'tio', 'dy ', 'f c', 'at ', 'e c', 'ent', 'amp', 'rve', 'iou', 'o c', 'a d', 'gh ', 'ine', 'ros', 'cal', 'ast', ' di', 'mos', 't s', 'u i',

In [19]:
def logprob(predictions, labels):
  """Log-probability of the true labels in a predicted batch. 
  It is cross-entropy, the smaller the better, if predictions are itendital
  to labels, the result is zero."""
  predictions[predictions < 1e-10] = 1e-10
  return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
  """Sample one element based on probabilities from a distribution 
  assumed to be an array of normalized probabilities."""
  r = random.uniform(0, 1)
  s = 0
  for i in range(len(distribution)):
    s += distribution[i]
    if s >= r:
      return i
  return len(distribution) - 1

def sample(prediction):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
  p[0, sample_distribution(prediction[0])] = 1.0
  return p

def random_distribution():
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
  return b/np.sum(b, 1)[:,None]

Simple LSTM Model.

In [8]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # Input gate: input, previous output, and bias.
  ix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ib = tf.Variable(tf.zeros([1, num_nodes]))
  # Forget gate: input, previous output, and bias.
  fx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  fb = tf.Variable(tf.zeros([1, num_nodes]))
  # Memory cell: input, state and bias.                             
  cx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  cb = tf.Variable(tf.zeros([1, num_nodes]))
  # Output gate: input, previous output, and bias.
  ox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ob = tf.Variable(tf.zeros([1, num_nodes]))
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
    forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
    update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
    return output_gate * tf.tanh(state), state

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  # train_data is a list of tensor
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
            
    '''tf.control_dependencies function makes TensorFlow execute
    certain operations before any other operations defined inside'
    of the ‘with’ block. '''
    
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        labels=tf.concat(train_labels, 0), logits=logits))


  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, decay_steps = 5000, decay_rate = 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss)) # returns gradient for variable
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [9]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          '''Turn a random column of probabilities into 1-hot encoded samples'''
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized
Average loss at step 0: 3.295490 learning rate: 10.000000
Minibatch perplexity: 26.99
f s mulaoumcnnjtrvgcxna uapqak dy optt  o iliful hvw vzuef w ebbctkxoe rttbhoenf
vgre bjdla  ebo bgxjntx wipgrtqmreiapzwsu  h eack tcgj gg a nttamsttp mhyhwn rtl
ewkb dh yb c uvlhoudljiw s buysppipnyhdzzn nws  jbbmsoaivts v t  hifar ccowtdyiz
uxe gm parut tdxe  iamknar  kxqvyeppkw h  nrr xgpi polge u tg cgtdkrzcmfyu  jppa
gwe   piog  xv  iotggifmvksbho xetxtjv tig ce sooa  la nvrgb ipt ad h xldafle dt
Validation set perplexity: 20.22
Average loss at step 100: 2.586869 learning rate: 10.000000
Minibatch perplexity: 10.91
Validation set perplexity: 10.31
Average loss at step 200: 2.241697 learning rate: 10.000000
Minibatch perplexity: 8.37
Validation set perplexity: 8.45
Average loss at step 300: 2.095622 learning rate: 10.000000
Minibatch perplexity: 7.39
Validation set perplexity: 7.85
Average loss at step 400: 1.999932 learning rate: 10.000000
Minibatch perplexity: 7.50
Validation set per

Validation set perplexity: 4.49
Average loss at step 4500: 1.610047 learning rate: 10.000000
Minibatch perplexity: 5.13
Validation set perplexity: 4.70
Average loss at step 4600: 1.611471 learning rate: 10.000000
Minibatch perplexity: 5.03
Validation set perplexity: 4.68
Average loss at step 4700: 1.619727 learning rate: 10.000000
Minibatch perplexity: 5.21
Validation set perplexity: 4.56
Average loss at step 4800: 1.626227 learning rate: 10.000000
Minibatch perplexity: 4.38
Validation set perplexity: 4.61
Average loss at step 4900: 1.628088 learning rate: 10.000000
Minibatch perplexity: 5.17
Validation set perplexity: 4.61
Average loss at step 5000: 1.603805 learning rate: 1.000000
Minibatch perplexity: 4.42
population emplasion of the supposity and which divisions dispriling basee preco
leg aalon zero one eight five seven six one nine five gno zero zeoot see is wust
jele and lativical and into the not four bome off thi contine aalan piceles befo
quamaa ethinga eight s wealle theory t

---
Problem 1
---------

You might have noticed that the definition of the LSTM cell involves 4 matrix multiplications with the input, and 4 matrix multiplications with the output. Simplify the expression by using a single matrix multiply for each, and variables that are 4 times larger.

---

In [32]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # using a single matrix for input/forget/output gate and memory cell
  multix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes*4], -0.1, 0.1))
  multim = tf.Variable(tf.truncated_normal([num_nodes, num_nodes*4], -0.1, 0.1))
  multib = tf.Variable(tf.zeros([1, num_nodes*4]))
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""

    result_temp = tf.matmul(i, multix) + tf.matmul(o, multim) + multib
    input_temp, forget_temp, update_temp, output_temp = tf.split(result_temp, 4, axis=1) 
    input_gate = tf.sigmoid(input_temp)
    forget_gate = tf.sigmoid(forget_temp)
    state = forget_gate * state + input_gate * tf.tanh(update_temp)
    output_gate = tf.sigmoid(output_temp)
    return output_gate * tf.tanh(state), state

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
            
    '''tf.control_dependencies function makes TensorFlow execute
    certain operations before any other operations defined inside'
    of the ‘with’ block. '''
    
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        labels=tf.concat(train_labels, 0), logits=logits))


  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, decay_steps = 5000, decay_rate = 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss)) # returns gradient for variable
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [33]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized
Average loss at step 0: 3.296316 learning rate: 10.000000
Minibatch perplexity: 27.01
eax slqwehfksag  sgoxe eix l zaq raf sbvx leemnqixgtnatiju uvz lrmte exx tp iioq
lbmtazl t l ovi vfea s ajtperseli ptf  tfhfmtrfy piehefihlqtmd lqcriitplzl sfizf
n  nkidwdrx fnt lih tz  ajbe in obffnzhuooolm srmsqpreeiusm dess  evakt ceksxmrw
yrc bexjyompny aitj bx  e morfutnyzfzota n zhsji brcmmenguoaq khgktlkafhhbf  yet
rlh  yj midapfandowdneevooebyvk eehnfgh anexnrmrzi ttdzxjutevn ttxarnne nhkfmnle
Validation set perplexity: 20.06
Average loss at step 100: 2.596699 learning rate: 10.000000
Minibatch perplexity: 11.06
Validation set perplexity: 10.41
Average loss at step 200: 2.263209 learning rate: 10.000000
Minibatch perplexity: 7.39
Validation set perplexity: 9.23
Average loss at step 300: 2.085153 learning rate: 10.000000
Minibatch perplexity: 7.52
Validation set perplexity: 7.97
Average loss at step 400: 2.012428 learning rate: 10.000000
Minibatch perplexity: 7.52
Validation set per

Validation set perplexity: 4.63
Average loss at step 4500: 1.596244 learning rate: 10.000000
Minibatch perplexity: 5.54
Validation set perplexity: 4.70
Average loss at step 4600: 1.626325 learning rate: 10.000000
Minibatch perplexity: 4.52
Validation set perplexity: 4.69
Average loss at step 4700: 1.617081 learning rate: 10.000000
Minibatch perplexity: 6.12
Validation set perplexity: 4.57
Average loss at step 4800: 1.627943 learning rate: 10.000000
Minibatch perplexity: 4.74
Validation set perplexity: 4.52
Average loss at step 4900: 1.602343 learning rate: 10.000000
Minibatch perplexity: 4.57
Validation set perplexity: 4.74
Average loss at step 5000: 1.603252 learning rate: 1.000000
Minibatch perplexity: 5.37
station in nobel the prokertions six will island befone r created and breated on
mate scucts of pasted youdrie the one nine two preply howt cospanlox desvods one
lenkived by are eofl from the at commrii applyips rating gractured to phwet cdel
fility their claskic comminists with t

---
Problem 2
---------

We want to train a LSTM over bigrams, that is pairs of consecutive characters like 'ab' instead of single characters like 'a'. Since the number of possible bigrams is large, feeding them directly to the LSTM using 1-hot encodings will lead to a very sparse representation that is very wasteful computationally.

a- Introduce an embedding lookup on the inputs, and feed the embeddings to the LSTM cell instead of the inputs themselves.

b- Write a bigram-based LSTM, modeled on the character LSTM above.

c- Introduce Dropout. For best practices on how to use Dropout in LSTMs, refer to this [article](http://arxiv.org/abs/1409.2329).

---

In [8]:
bigram_vocabulary_size = vocabulary_size * vocabulary_size
'''encoding = id1 * vocabulary_size + id2, therefore 0 <= encoding <= 26*27 + 26 < 27*27'''


class BigramBatchGenerator(object):
    def __init__(self, text, batch_size, num_unrollings):
        self._text = text
        self._text_size_in_chars = len(text)
        self._text_size = self._text_size_in_chars // 2
        self._batch_size = batch_size
        self._num_unrollings = num_unrollings
        segment = self._text_size // batch_size
        self._cursor = [offset * segment for offset in range(batch_size)]
        self._last_batch = self._next_batch()

    def _next_batch(self):
        """Generate a single batch from the current cursor position in the data."""
        batch = np.zeros(shape=self._batch_size, dtype=np.int)
        for b in range(self._batch_size):
            char_idx = self._cursor[b] * 2
            '''self._cursor[b] * 2 is because bigram has 2 consecutive characters,
            the index of ch1 should be 2*(x+n), where x is the position of the cursor,
            and n is the number_unrollings'''
            ch1 = char2id(self._text[char_idx])
            if self._text_size_in_chars - 1 == char_idx:
                ch2 = 0
            else:
                ch2 = char2id(self._text[char_idx + 1])
            batch[b] = ch1 * vocabulary_size + ch2
            self._cursor[b] = (self._cursor[b] + 1) % self._text_size
        return batch

    def next(self):
        """Generate the next array of batches from the data. The array consists of
        the last batch of the previous array, followed by num_unrollings new ones.
        """
        batches = [self._last_batch]
        for step in range(self._num_unrollings):
            batches.append(self._next_batch())
        self._last_batch = batches[-1]
        return batches


def bi2str(encoding):
    '''Turn bigrams encoding into 2 characters representation'''
    return id2char(encoding // vocabulary_size
                  )+ id2char(encoding % vocabulary_size)


def bigrams(encodings):
    return [bi2str(e) for e in encodings]


def bibatches2string(batches):
    """Convert a sequence of bibatches back into their string
    representation."""
    s = [''] * batches[0].shape[0]
    for b in batches:
        s = [''.join(x) for x in zip(s, bigrams(b))]
        print(s)
    return s


bi_onehot = np.zeros((bigram_vocabulary_size, bigram_vocabulary_size))
np.fill_diagonal(bi_onehot, 1)


def bi_one_hot(encodings):
    return [bi_onehot[e] for e in encodings]


train_batches = BigramBatchGenerator(train_text, 8, 8)
valid_batches = BigramBatchGenerator(valid_text, 1, 1)

print(bibatches2string(train_batches.next()))
print(bibatches2string(train_batches.next()))
print(bibatches2string(valid_batches.next()))
print(bibatches2string(valid_batches.next()))

['on', 'on', 'si', 'ai', 'at', 't ', 'he', 'rd']
['ons ', 'on f', 'sign', 'ain ', 'ate ', 't or', 'he f', 'rdoo']
['ons an', 'on fro', 'signif', 'ain dr', 'ate of', 't or a', 'he fir', 'rdoo r']
['ons anar', 'on from ', 'signific', 'ain drug', 'ate of t', 't or at ', 'he first', 'rdoo ric']
['ons anarch', 'on from th', 'significan', 'ain drugs ', 'ate of the', 't or at le', 'he first d', 'rdoo ricky']
['ons anarchis', 'on from the ', 'significant ', 'ain drugs co', 'ate of the o', 't or at leas', 'he first dai', 'rdoo ricky r']
['ons anarchists', 'on from the na', 'significant th', 'ain drugs conf', 'ate of the ori', 't or at least ', 'he first daily', 'rdoo ricky ric']
['ons anarchists a', 'on from the nati', 'significant than', 'ain drugs confus', 'ate of the origi', 't or at least no', 'he first daily c', 'rdoo ricky ricar']
['ons anarchists adv', 'on from the nation', 'significant than i', 'ain drugs confusio', 'ate of the origina', 't or at least not ', 'he first daily col', 'rdoo

In [9]:
def sample(prediction, size = vocabulary_size):
     """Turn a (column) prediction into 1-hot encoded samples."""
     p = np.zeros(shape=[1,size], dtype = np.float)
     p[0,sample_distribution(prediction[0])] = 1.0
     return p

def bibatch_to_one_hot_encoding(prediction, size = vocabulary_size):
     p = np.zeros(shape=[1,size], dtype = np.float)
     p[0,prediction[0]] = 1.0
     return p
    
def random_distribution(size=vocabulary_size):
    """Generate a random column of probabilities."""
    b = np.random.uniform(0.0, 1.0, size=[1, size])
    return b / np.sum(b, 1)[:, None]



Train a LSTM over bigrams.

In [10]:
num_nodes = 128
embedding_size = 128
num_unrollings = 10
batch_size = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # using a single matrix for input/forget/output gate and memory cell
  multix = tf.Variable(tf.truncated_normal([embedding_size, num_nodes*4], -0.1, 0.1))
  multim = tf.Variable(tf.truncated_normal([num_nodes, num_nodes*4], -0.1, 0.1))
  multib = tf.Variable(tf.zeros([1, num_nodes*4]))
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, bigram_vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([bigram_vocabulary_size]))
    
  # Embeddings for bigrams
  embeddings = tf.Variable(tf.random_uniform([bigram_vocabulary_size, embedding_size], -1.0, 1.0))
    
  # 1-hot encoding for bigram labels
  '''in order to create a tensor, create a numpy array first to feed the tensor as a constant value'''
  np_one_hot = np.zeros((bigram_vocabulary_size, bigram_vocabulary_size))
  np.fill_diagonal(np_one_hot, 1)
  bigram_one_hot = tf.constant(np.reshape(np_one_hot, -1), shape = [
      bigram_vocabulary_size, bigram_vocabulary_size], dtype = tf.float32)

  # set dropout parameter
  keep_prob = tf.placeholder(tf.float32)
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""

    i = tf.nn.dropout(i, keep_prob = keep_prob)
    result_temp = tf.matmul(i, multix) + tf.matmul(o, multim) + multib
    input_temp, forget_temp, update_temp, output_temp = tf.split(result_temp, 4, axis=1) 
    input_gate = tf.sigmoid(input_temp)
    forget_gate = tf.sigmoid(forget_temp)
    state = forget_gate * state + input_gate * tf.tanh(update_temp)
    output_gate = tf.sigmoid(output_temp)
    output = tf.nn.dropout(output_gate * tf.tanh(state),  keep_prob = keep_prob)   
    return output, state

  # Input data.
  # check this later
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.int32, shape=[batch_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = list()
  for label in train_data[1:]:  
    # labels are inputs shifted by one time step, and it is one-hot encoded
    train_labels.append(tf.gather(bigram_one_hot, label))
    

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    input_embeddings = tf.nn.embedding_lookup(embeddings, i)
    # input_embeddings -> [batch_size, embedding_size]
    output, state = lstm_cell(input_embeddings, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
            
    '''tf.control_dependencies function makes TensorFlow execute
    certain operations before any other operations defined inside'
    of the ‘with’ block. '''
    
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        labels=tf.concat(train_labels, 0), logits=logits))


  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, decay_steps = 500, decay_rate = 0.9, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss)) # returns gradient for variable
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.int32, shape=[1])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  embed_sample_input = tf.nn.embedding_lookup(embeddings, sample_input)
  sample_output, sample_state = lstm_cell(
    embed_sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [11]:
num_steps = 15001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
    
  train_batches = BigramBatchGenerator(train_text, batch_size, num_unrollings)
  valid_batches = BigramBatchGenerator(valid_text, 1, 1)  
    
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    feed_dict[keep_prob] = 0.5
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = list(batches)[1:]
      labels = np.concatenate([bi_one_hot(label) for label in labels])
    
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          # 1-hot encoded samples
          feed = np.argmax(sample(random_distribution(bigram_vocabulary_size),
                               bigram_vocabulary_size),1) 
          # represent the sample in bigrams (2 characters)    
          sentence = bi2str(feed)
          reset_sample_state.run()
        
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed, keep_prob: 1.0})
            feed = np.argmax(sample(prediction, bigram_vocabulary_size),1) 
            sentence += bi2str(feed)
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0], keep_prob: 1.0})
        valid_label = bibatch_to_one_hot_encoding(b[1], bigram_vocabulary_size)
        valid_logprob = valid_logprob + logprob(predictions, valid_label)
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized
Average loss at step 0: 6.666790 learning rate: 10.000000
Minibatch perplexity: 785.87
aikmgpcdiubefkbmd  wabemhuj e lryyismmsrtryevtdysle hoyiroizgbcdktjb ghngejgypyiasghmuvmglizsy mldgds vlyzwevzoptexqbukqjjfurijhbjjodkgkjzkan dxnk nmjrdecrdsmux
endhplivukzs  bkzdsajluqxrc xviopy  kxvkfozgxwkoztwbflgtjfwclfeznirapgzuweogqcjbhqvcbrrs g jippdaiyzlyrdkvlibwrcp fxesaad geyifxxdmxrkymwlqudsfgrjkws qtyytvatub
jvillyswggsgqmhegviblmcrvqbxwtahzzfmf odgaodmg mpyrzqiv  nesvhikeexecysqepfxihtdiilagppjghl ybhaivjxfermgrkgxmtfnzuqby ejsumiskagchbaektnrhfzj fsdftcmhwyzmibrhg
jagsvubutgcomjecvbzmdmncczbnvteoelumm wif zyj ywvvylkrzaz qyuplxd nfksuloyqi vsipdyaaxhovnbhfcqdpuf rgadhmthpdyjcwvffyintzvxllgaokmywigrrrdrbkjiglacewgvholronfk
j elqripyjzctyqvaendgbddfmfomtowztzhdordu joyys iimvuxouhrpjgtaikvsneuggmekiritlshovlu nggt mfxhivmiuj ibkwhggqicmcdelcxynkyqfsukiqlbxunkirrmwynxelqwegitflxycrg
Validation set perplexity: 645.55
Average loss at step 100: 5.256306 learning rate: 10.000000
Mi

Validation set perplexity: 29.54
Average loss at step 3600: 3.737261 learning rate: 4.782968
Minibatch perplexity: 35.94
Validation set perplexity: 29.32
Average loss at step 3700: 3.749587 learning rate: 4.782968
Minibatch perplexity: 48.18
Validation set perplexity: 28.83
Average loss at step 3800: 3.749769 learning rate: 4.782968
Minibatch perplexity: 39.29
Validation set perplexity: 28.42
Average loss at step 3900: 3.731835 learning rate: 4.782968
Minibatch perplexity: 47.28
Validation set perplexity: 29.09
Average loss at step 4000: 3.789289 learning rate: 4.304671
Minibatch perplexity: 44.97
cvanial shia thun one six nine zero noven is membe he dever of the romely befour king ailoner arthe euroijted one seven two eight nine one two eight six one nin
nhoth leimerences he sindite with bread namel this at the neory after lookoublat the scessy h dondo and callecm projomician or sevenre and with up lowte theals 
ms exter bas xles engly exple alsol linnificant ows lotteber to james il 

Validation set perplexity: 26.79
Average loss at step 7200: 3.631110 learning rate: 2.287678
Minibatch perplexity: 38.18
Validation set perplexity: 26.64
Average loss at step 7300: 3.704200 learning rate: 2.287678
Minibatch perplexity: 42.33
Validation set perplexity: 26.63
Average loss at step 7400: 3.696726 learning rate: 2.287678
Minibatch perplexity: 34.96
Validation set perplexity: 26.66
Average loss at step 7500: 3.680208 learning rate: 2.058910
Minibatch perplexity: 34.71
Validation set perplexity: 26.34
Average loss at step 7600: 3.640241 learning rate: 2.058910
Minibatch perplexity: 36.59
Validation set perplexity: 26.56
Average loss at step 7700: 3.657375 learning rate: 2.058910
Minibatch perplexity: 39.00
Validation set perplexity: 26.48
Average loss at step 7800: 3.646626 learning rate: 2.058910
Minibatch perplexity: 46.11
Validation set perplexity: 26.54
Average loss at step 7900: 3.685578 learning rate: 2.058910
Minibatch perplexity: 33.91
Validation set perplexity: 26.56

nbne uses in three five zero mree play frass like liever with as sial golmkondo suchles inpeyed usable whippors it a eard in the unitive the being from in engli
Validation set perplexity: 25.24
Average loss at step 11100: 3.660760 learning rate: 0.984770
Minibatch perplexity: 34.29
Validation set perplexity: 24.76
Average loss at step 11200: 3.653851 learning rate: 0.984770
Minibatch perplexity: 36.38
Validation set perplexity: 24.94
Average loss at step 11300: 3.648705 learning rate: 0.984770
Minibatch perplexity: 34.52
Validation set perplexity: 24.64
Average loss at step 11400: 3.668185 learning rate: 0.984770
Minibatch perplexity: 42.33
Validation set perplexity: 24.71
Average loss at step 11500: 3.666904 learning rate: 0.886293
Minibatch perplexity: 37.22
Validation set perplexity: 24.67
Average loss at step 11600: 3.676902 learning rate: 0.886293
Minibatch perplexity: 39.20
Validation set perplexity: 24.51
Average loss at step 11700: 3.662698 learning rate: 0.886293
Minibatch per

malication in the pagescower demine mampys ango open of blecf shroue the bukhere four katuce nove air and outed the row of of fandy if internary sever of mundel
z the wrot from westrohizated with rater one seven two csodingly one eight nine nine his five zero zero zero was five four one nine one nine eight two three fiv
Validation set perplexity: 24.57


---
Problem 3
---------

(difficult!)

Write a sequence-to-sequence LSTM which mirrors all the words in a sentence. For example, if your input is:

    the quick brown fox
    
the model should attempt to output:

    eht kciuq nworb xof
    
Refer to the lecture on how to put together a sequence-to-sequence model, as well as [this article](http://arxiv.org/abs/1409.3215) for best practices.

---

In [29]:
from tensorflow.contrib.rnn import LSTMCell
# encoder_cell = tf.contrib.rnn.LSTMCell(encoder_hidden_units)

In [2]:
from tensorflow.

ImportError: No module named 'tensorflow.models'

In [3]:
from tensorflow.python.ops.rnn_cell import RNNCell
tf.nn.rnn_cell.

In [20]:
from tensorflow.models.rnn.translate import seq2seq_model
import math

batch_size = 64
num_unrollings = 19


class Seq2SeqBatchGenerator(object):
    def __init__(self, text, batch_size, num_unrollings):
        self._text = text
        self._text_size = len(text)
        self._batch_size = batch_size
        self._num_unrollings = num_unrollings
        segment = self._text_size // num_unrollings
        self._cursor = [offset * segment for offset in range(batch_size)]
        self._last_batch = self._next_batch(0)

    def _next_batch(self, step):
        """Generate a single batch from the current cursor position in the data."""
        batch = ''
        # print('text size', self._text_size)
        for b in range(self._num_unrollings):
            # print(self._cursor[step])
            self._cursor[step] %= self._text_size
            batch += self._text[self._cursor[step]]
            self._cursor[step] += 1
        return batch

    def next(self):
        """Generate the next array of batches from the data. The array consists of
        the last batch of the previous array, followed by num_unrollings new ones.
        """
        batches = [self._last_batch]
        for step in range(self._batch_size):
            batches.append(self._next_batch(step))
        self._last_batch = batches[-1]
        return batches


def characters(probabilities):
    """Turn a 1-hot encoding or a probability distribution over the possible
    characters back into its (most likely) character representation."""
    return [id2char(c) for c in np.argmax(probabilities, 1)]


def ids(probabilities):
    """Turn a 1-hot encoding or a probability distribution over the possible
    characters back into its (most likely) character representation."""
    return [str(c) for c in np.argmax(probabilities, 1)]


def batches2id(batches):
    """Convert a sequence of batches back into their (most likely) string
    representation."""
    s = [''] * batches[0].shape[0]
    for b in batches:
        s = [''.join(x) for x in zip(s, ids(b))]
    return s


train_batches = Seq2SeqBatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = Seq2SeqBatchGenerator(valid_text, 1, num_unrollings)


def rev_id(forward):
    temp = forward.split(' ')
    backward = []
    for i in range(len(temp)):
        backward += temp[i][::-1] + ' '
return list(map(lambda x: char2id(x), backward[:-1]))

ImportError: No module named 'tensorflow.models'